In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from pathlib import Path 
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"

def eda_correlation_analysis(data_path, output_plot_path, target_column_name="Rowing distance", correlation_threshold=0.8): # 默认目标列名改为小写d
    """
    Performs correlation analysis on input features.

    Args:
        data_path (str): Path to the Excel data file.
        output_plot_path (str): Directory to save the output plots.
        target_column_name (str): Name of the target variable column.
        correlation_threshold (float): Absolute correlation value above which pairs are considered highly correlated.
    """
    # --- 1. 数据加载和准备 ---
    try:
        print(f"正在加载数据从: {data_path}")
        df = pd.read_excel(data_path)
    except FileNotFoundError:
        print(f"错误: 文件未找到 {data_path}")
        return None, None # 返回 None 表示失败
    except Exception as e:
        print(f"加载数据时发生错误: {e}")
        return None, None # 返回 None 表示失败

    print("数据加载成功。数据前5行:")
    print(df.head())
    print(f"\n数据形状: {df.shape}")

    # 确保目标列存在
    if target_column_name not in df.columns:
        actual_last_col_name = df.columns[-1]
        print(f"警告: 指定的目标列名 '{target_column_name}' 未在数据列中找到。")
        if target_column_name.lower() == actual_last_col_name.lower():
             print(f"检测到大小写不匹配，将使用实际列名 '{actual_last_col_name}' 作为目标。")
             target_column_name = actual_last_col_name
        else:
            print(f"将尝试使用最后一列 '{actual_last_col_name}' 作为目标列。请确认这是否正确。")
            target_column_name = actual_last_col_name
    
    if target_column_name not in df.columns: # 再次检查，如果最后一列也不是，则严重错误
        print(f"错误: 无法确定目标列。提供的 '{target_column_name}' 和最后一列都不是有效的列名。")
        return None, None


    X = df.drop(columns=[target_column_name])
    # y = df[target_column_name] # 目标变量，当前分析中未使用，但已分离

    print(f"\n输入特征数量: {X.shape[1]}")
    print(f"识别出的目标变量: {target_column_name}")

    # --- 2. 计算输入特征之间的相关系数矩阵 ---
    print("\n正在计算相关系数矩阵 (仅针对输入特征)...")
    corr_matrix = X.corr()

    # --- 3. 可视化相关系数矩阵 (热力图) ---
    print("正在生成热力图...")
    try:
        # 尝试使用一个更兼容的seaborn样式，如果仍然报错，可以注释掉下面这行
        plt.style.use('seaborn-v0_8-whitegrid')
    except OSError:
        print("警告: 无法加载 'seaborn-v0_8-whitegrid' 样式。将使用matplotlib默认样式。")
        # 如果需要，可以在这里设置一些基本的rcParams来美化，例如：
        # plt.rcParams['figure.facecolor'] = 'white'
        # plt.rcParams['axes.edgecolor'] = '.8'
        # plt.rcParams['grid.color'] = '.9'


    plt.figure(figsize=(max(15, X.shape[1]*0.45), max(15, X.shape[1]*0.4))) # 略微调整了大小计算

    # 使用发散型色板，类似于医学期刊风格 (例如，红-白-蓝)
    cmap = sns.diverging_palette(250, 15, s=80, l=55, n=9, center="light", as_cmap=True) # 调整了饱和度和亮度

    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

    sns.heatmap(corr_matrix,
                mask=mask,
                cmap=cmap,
                vmin=-1, vmax=1, center=0,
                square=True,
                linewidths=.3, # 减小线宽
                cbar_kws={"shrink": .75, "label": "Correlation Coefficient", "aspect": 30}, # 调整颜色条长宽比
                annot=False,
                fmt=".2f")

    plt.title(f'Correlation Matrix of Input Features ({X.shape[1]} features)', fontsize=18, pad=20) # 增加标题字体和间距
    # plt.xticks(rotation=90, ha='right', fontsize=9) # 增加X轴标签字体
    # plt.yticks(rotation=0, fontsize=9) # 增加Y轴标签字体
    plt.xticks(rotation=45, ha='right', fontsize=16)   # 横坐标倾斜 45°
    plt.yticks(rotation=0, ha='right', fontsize=16)   # 纵坐标倾斜 45°
    plt.tight_layout(pad=1.5) # 调整整体布局，增加边距

    os.makedirs(output_plot_path, exist_ok=True)
    heatmap_filename = os.path.join(output_plot_path, "input_features_correlation_heatmap.png")

    try:
        plt.savefig(heatmap_filename, dpi=300, bbox_inches='tight')
        print(f"\n热力图已保存至: {heatmap_filename}")
    except Exception as e:
        print(f"保存热力图时发生错误: {e}")
    plt.show()

    # --- 4. 识别并输出高度相关的特征对 ---
    print(f"\n寻找绝对相关系数 > {correlation_threshold} 的特征对:")
    highly_correlated_pairs = []
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    for column in upper_tri.columns:
        for index in upper_tri.index:
            correlation_value = upper_tri.loc[index, column]
            if pd.notna(correlation_value) and abs(correlation_value) > correlation_threshold:
                highly_correlated_pairs.append(((index, column), correlation_value))

    if highly_correlated_pairs:
        print(f"共找到 {len(highly_correlated_pairs)} 对高度相关的特征:")
        highly_correlated_pairs.sort(key=lambda x: abs(x[1]), reverse=True)
        for pair_info in highly_correlated_pairs:
            features, value = pair_info
            print(f"  特征对: {features[0]} 和 {features[1]}, 相关系数: {value:.3f}")
    else:
        print(f"未找到绝对相关系数高于 {correlation_threshold} 的强相关特征对。")

    print("\nEDA相关性分析完成。")
    return corr_matrix, highly_correlated_pairs

if __name__ == '__main__':
    # --- 用户配置 ---
    data_file_path = DATA_DIR / "development_set.xlsx"
    plots_output_directory = OUTPUT_DIR
    
    # !!! 重要: 请根据你的Excel文件，确认最后一列目标变量的准确列名(区分大小写) !!!
    # 根据你的错误日志，实际列名似乎是 'Rowing distance' (小写d)
    target_col = "Rowing distance" 
    
    correlation_abs_threshold = 0.8

    # --- 执行分析 ---
    correlation_matrix, correlated_pairs = eda_correlation_analysis(
        data_path=data_file_path,
        output_plot_path=plots_output_directory,
        target_column_name=target_col,
        correlation_threshold=correlation_abs_threshold
    )
    
    if correlated_pairs is not None: # 检查是否成功执行
        try:
            correlated_pairs_df = pd.DataFrame(
                [(p[0][0], p[0][1], p[1]) for p in correlated_pairs],
                columns=['Feature 1', 'Feature 2', 'Correlation Coefficient']
            )
            correlated_pairs_filename = os.path.join(plots_output_directory, "highly_correlated_feature_pairs.csv")
            correlated_pairs_df.to_csv(correlated_pairs_filename, index=False)
            print(f"\n高度相关的特征对列表已保存至: {correlated_pairs_filename}")
        except Exception as e:
            print(f"保存高度相关特征对列表时发生错误: {e}")

In [ ]:
# 基于原始数据 - 46
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os # 导入os模块用于创建文件夹

from pathlib import Path 
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"

import xgboost as xgb
try:
    print("尝试使用GPU初始化一个简单的XGBoost模型...")
    # 尝试初始化一个使用gpu_hist的模型
    # 如果这一步没有因为GPU问题报错，说明XGBoost至少能识别并尝试使用GPU
    temp_model = xgb.XGBClassifier(tree_method='gpu_hist')
    # 或者对于回归任务
    # temp_model = xgb.XGBRegressor(tree_method='gpu_hist')
    print("XGBoost GPU tree_method ('gpu_hist') 初始化成功。")
    # 你甚至可以尝试一个非常小的数据集进行拟合来确认
    # import numpy as np
    # temp_model.fit(np.array([[1],[2]]), np.array([0,1]))
    # print("简单模型在GPU上拟合成功。")
except xgb.core.XGBoostError as e:
    print(f"XGBoost GPU 初始化或测试失败: {e}")
    print("请检查：")
    print("1. XGBoost是否安装了GPU版本 (例如，通过 'conda install py-xgboost-gpu' 或确保pip安装的是GPU编译版)。")
    print("2. NVIDIA驱动程序是否已正确安装且为最新。")
    print("3. NVIDIA CUDA Toolkit是否已正确安装并与驱动程序/XGBoost版本兼容。")
    print("4. CUDA相关的环境变量路径是否设置正确。")
except Exception as e:
    print(f"发生其他错误在尝试GPU初始化时: {e}")
# --- 配置部分 ---
# !!! 注意: 请确保将此路径更改为你的实际文件路径 !!!
data_path = DATA_DIR / "development_set.xlsx" 
# !!! 注意: 请确保 'target' 是你数据中实际的目标变量列名 !!!
target_column_name = 'Rowing distance' # 如果你的目标列名不同 (例如 'Rowing_Distance'), 请相应修改

# 图表导出路径
output_plot_path = OUTPUT_DIR
# 创建导出路径 (如果不存在)
os.makedirs(output_plot_path, exist_ok=True)


# --- 数据加载 ---
original_data = pd.read_excel(data_path)
X_original = original_data.drop(columns=[target_column_name])
y_original = original_data[target_column_name]

# --- XGBoost 超参数调优 ---
# 准备 RandomizedSearchCV 的参数网格
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'subsample': [0.7, 0.8, 0.9, 1],
    'colsample_bytree': [0.7, 0.8, 0.9, 1],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.01, 0.1, 0.5, 1],
    'reg_lambda': [0.5, 1, 1.5, 2]
}

# 初始化 XGBoost 回归器模型
# 修改: 添加 tree_method='gpu_hist' 以尝试使用GPU; objective='reg:squarederror' 是常见的回归目标函数
# 注意: GPU加速需要兼容的硬件和软件环境
xgb_regressor_base = XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', random_state=42)

print("开始超参数调优 (RandomizedSearchCV)...")
# 在原始数据集上执行 Randomized Search (5折交叉验证) 寻找最佳参数
random_search_original = RandomizedSearchCV(
    estimator=xgb_regressor_base,
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=2,
    random_state=42,
    n_jobs=1 # 使用所有可用CPU核心进行交叉验证的并行处理
)
random_search_original.fit(X_original, y_original)

best_params_original = random_search_original.best_params_
print("最佳参数 (原始数据):", best_params_original)
print(f"最佳交叉验证得分 ({random_search_original.scoring}): {random_search_original.best_score_}")


# --- K折交叉验证评估 ---
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results_original_metrics = []
train_losses = [] # 每折的训练损失 (MAE)
test_losses = []  # 每折的测试损失 (MAE)

print("\n使用最佳参数进行K折交叉验证评估:")
for fold, (train_index, test_index) in enumerate(kf.split(X_original), 1):
    X_train, X_test = X_original.iloc[train_index], X_original.iloc[test_index]
    y_train, y_test = y_original.iloc[train_index], y_original.iloc[test_index]

    # 使用最佳参数初始化 XGBoost 回归器模型
    # 修改: 添加 tree_method='gpu_hist'
    xgb_regressor_fold = XGBRegressor(**best_params_original, objective='reg:squarederror', tree_method='gpu_hist', random_state=42)

    eval_set = [(X_train, y_train), (X_test, y_test)]
    xgb_regressor_fold.fit(X_train, y_train, eval_metric='mae', eval_set=eval_set, early_stopping_rounds=10, verbose=False)

    fold_results_eval = xgb_regressor_fold.evals_result()
    train_losses.append(fold_results_eval['validation_0']['mae'])
    test_losses.append(fold_results_eval['validation_1']['mae'])

    y_pred_test = xgb_regressor_fold.predict(X_test)
    y_pred_train = xgb_regressor_fold.predict(X_train)

    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_test, y_pred_test)

    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y_train, y_pred_train)

    print(f"Fold {fold} Results: Train MAE: {mae_train:.4f}, Train R2: {r2_train:.4f} | Test MAE: {mae_test:.4f}, Test R2: {r2_test:.4f}")
    results_original_metrics.append((mae_test, mse_test, rmse_test, r2_test, mae_train, mse_train, rmse_train, r2_train))

# --- 计算平均性能 ---
avg_mae_test = np.mean([res[0] for res in results_original_metrics])
avg_mse_test = np.mean([res[1] for res in results_original_metrics])
avg_rmse_test = np.mean([res[2] for res in results_original_metrics])
avg_r2_test = np.mean([res[3] for res in results_original_metrics])

avg_mae_train = np.mean([res[4] for res in results_original_metrics])
avg_mse_train = np.mean([res[5] for res in results_original_metrics])
avg_rmse_train = np.mean([res[6] for res in results_original_metrics])
avg_r2_train = np.mean([res[7] for res in results_original_metrics])

print("\nOriginal Data - Average Test Performance:")
print(f"  MAE: {avg_mae_test:.4f}")
print(f"  MSE: {avg_mse_test:.4f}")
print(f"  RMSE: {avg_rmse_test:.4f}")
print(f"  R2 Score: {avg_r2_test:.4f}")

print("\nOriginal Data - Average Train Performance:")
print(f"  MAE: {avg_mae_train:.4f}")
print(f"  MSE: {avg_mse_train:.4f}")
print(f"  RMSE: {avg_rmse_train:.4f}")
print(f"  R2 Score: {avg_r2_train:.4f}")

# --- 绘制图表并导出 ---

# 绘制评估指标图 (MAE 和 R2)
metrics_plot_names_en = ['MAE', 'R2 Score'] # 英文指标名称
values_test_plot = [avg_mae_test, avg_r2_test]
values_train_plot = [avg_mae_train, avg_r2_train]

x_axis_plot = np.arange(len(metrics_plot_names_en))
plt.figure(figsize=(10, 6))
plt.bar(x_axis_plot - 0.2, values_train_plot, width=0.4, label='Train', align='center')
plt.bar(x_axis_plot + 0.2, values_test_plot, width=0.4, label='Test', align='center')
plt.xticks(x_axis_plot, metrics_plot_names_en)
plt.ylabel('Score') # Y轴标签改为英文
plt.title('Average Train vs. Test Set Evaluation Metrics (Original Data)') # 标题改为英文
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
# 导出图表
plot_filename_metrics = os.path.join(output_plot_path, "average_evaluation_metrics.png")
plt.savefig(plot_filename_metrics, dpi=300, bbox_inches='tight')
plt.show()


# 绘制每折的训练和测试损失 (MAE)
plt.figure(figsize=(14, 7))
for i in range(len(train_losses)):
    plt.plot(train_losses[i], label=f'Train Fold {i+1}', linestyle='-')
    plt.plot(test_losses[i], label=f'Test Fold {i+1}', linestyle='--')
plt.xlabel('Boosting Round') # X轴标签改为英文
plt.ylabel('Mean Absolute Error (MAE)') # Y轴标签改为英文
plt.title('Training and Testing MAE per Fold (Original Data)') # 标题改为英文
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
# 导出图表
plot_filename_loss = os.path.join(output_plot_path, "mae_loss_per_fold.png")
plt.savefig(plot_filename_loss, dpi=300, bbox_inches='tight')
plt.show()


# --- 过拟合检查 ---
r2_diff = avg_r2_train - avg_r2_test
print("\nOverfitting Check:")
# 这里的过拟合判断逻辑可以根据具体需求调整
if r2_diff > 0.1 and avg_r2_train > avg_r2_test: # 训练R2显著高于测试R2
    print(f"Warning: Model might be overfitting! Train R2 ({avg_r2_train:.2f}) is significantly higher than Test R2 ({avg_r2_test:.2f}). Difference: {r2_diff:.2f}")
elif avg_mae_test > 0 and avg_mae_train > 0 and (avg_mae_test - avg_mae_train) / avg_mae_train > 0.20 : # 测试MAE比训练MAE高出20%
     print(f"Warning: Model might be overfitting! Test MAE ({avg_mae_test:.2f}) is >20% higher than Train MAE ({avg_mae_train:.2f}). Relative difference: {((avg_mae_test - avg_mae_train) / avg_mae_train)*100:.2f}%.")
else:
    print(f"No strong signs of overfitting detected based on current thresholds. R2 Diff (Train-Test): {r2_diff:.2f}. MAE Train: {avg_mae_train:.2f}, MAE Test: {avg_mae_test:.2f}.")


# --- 特征重要性分析 ---
print("\nFeature Importance Analysis:")
# 修改: 添加 tree_method='gpu_hist'
final_model = XGBRegressor(**best_params_original, objective='reg:squarederror', tree_method='gpu_hist', random_state=42)
final_model.fit(X_original, y_original)

feature_importances = final_model.feature_importances_
importance_df = pd.DataFrame({'Feature': X_original.columns, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Feature Importances (High to Low):")
print(importance_df.head(min(20, len(X_original.columns)))) # 打印最重要的，最多20个

# 绘制特征重要性图
# 动态调整图像高度以容纳特征名称
num_features_to_plot = min(20, len(X_original.columns))
plot_height = max(6, num_features_to_plot * 0.4) # 根据特征数量调整高度
plt.figure(figsize=(10, plot_height))
plt.barh(importance_df['Feature'][:num_features_to_plot], importance_df['Importance'][:num_features_to_plot])
plt.xlabel('Feature Importance') # X轴标签改为英文
plt.ylabel('Feature') # Y轴标签改为英文
plt.title(f'Top {num_features_to_plot} Feature Importances') # 标题改为英文
plt.gca().invert_yaxis()
plt.tight_layout()
plt.grid(True, axis='x', linestyle='--', alpha=0.7) # 只显示X轴网格线
# 导出图表
plot_filename_importance = os.path.join(output_plot_path, "feature_importances原始.png")
plt.savefig(plot_filename_importance, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n所有图表已尝试保存至: {output_plot_path}")

In [ ]:
# 测试集多余指标剔除
import pandas as pd
import os

from pathlib import Path 
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"

# --- 用户配置 ---
# 输入文件路径和名称
input_folder_path = DATA_DIR
input_file_name = "final_test_set.xlsx"
full_input_path = DATA_DIR/ input_file_name

# 输出文件名
output_file_name = "final_test_set_selected_features.xlsx" # Changed output name slightly for clarity
full_output_path = DATA_DIR / output_file_name

# 需要保留的22个指标的准确名称列表 (21输入特征 + 1目标变量)
# !!! 请务必确保以下列表中的名称与你 "data.xlsx" 文件中第一行的标题完全一致 (包括大小写和空格) !!!
columns_to_keep = [
    "PARS – 3",  # 请注意这里的'–'是en-dash，如果Excel中是普通连字符'-', 可能需要修改
    "average vertical jump height before exercise",
    "Alcohol use in the last 30 days",
    "weight",
    "CHO",
    "Smoking frequency in the last 30 days",
    "Triceps skinfold",
    "Left-hand grip strength",
    "subgluteal thigh circumference",
    "hip circumference",
    "Age",
    "Previous meal time",
    "light sleep", # Ensure this is the exact column name in your Excel
    "RMSSD", # Ensure this is the exact column name for the HRV RMSSD component in your Excel
    "high blood pressure", # Ensure this is the exact column name
    "waist circumference",
    "hemoglobin", # Ensure this is the exact column name
    "Body water percentage", # Ensure this is the exact column name
    "blood lactate", # Ensure this is the exact name for baseline blood lactate
    "PRO", # Kept based on domain knowledge
    "Blood glucose", # Kept based on domain knowledge, ensure exact name for baseline
    "Rowing distance" # 目标变量
]

# --- 执行操作 ---
try:
    # 1. 读取原始Excel文件
    print(f"正在从以下路径加载原始数据: {full_input_path}")
    original_df = pd.read_excel(full_input_path)
    print("原始数据加载成功。")
    print(f"原始数据包含 {original_df.shape[0]} 行 和 {original_df.shape[1]} 列。")
    print("原始数据列名:", original_df.columns.tolist())

    # 2. 检查需要保留的列是否存在于原始数据中
    missing_columns = [col for col in columns_to_keep if col not in original_df.columns]
    if missing_columns:
        print("\n警告：以下指定的列在原始文件中未找到，它们将被忽略：")
        for col in missing_columns:
            print(f" - '{col}'")
        # 更新 columns_to_keep 列表，只包含实际存在的列
        columns_to_keep_actual = [col for col in columns_to_keep if col in original_df.columns]
        if not columns_to_keep_actual:
            print("错误：所有指定的保留列都不在原始文件中或指定的列均缺失。无法继续。")
            exit() # 或者 raise ValueError
    else:
        columns_to_keep_actual = columns_to_keep

    # 如果之前有列缺失或为了确认，打印实际将操作的列数
    expected_columns_count = 22
    if len(columns_to_keep_actual) < expected_columns_count:
        print(f"\n注意：预期保留 {expected_columns_count} 个指标，但由于部分指定列未找到，")
        print(f"实际将仅基于以下 {len(columns_to_keep_actual)} 个在文件中找到的指定指标进行操作。")
    elif len(columns_to_keep_actual) > expected_columns_count:
        print(f"\n注意：预期保留 {expected_columns_count} 个指标，但列表中指定了 {len(columns_to_keep_actual)} 个指标。")
        print(f"将按列表中指定的 {len(columns_to_keep_actual)} 个指标进行操作。")


    # 3. 筛选出指定的列
    print(f"\n正在筛选以下 {len(columns_to_keep_actual)} 个指标:")
    for col in columns_to_keep_actual:
        print(f" - '{col}'")
    
    selected_df = original_df[columns_to_keep_actual]
    print("\n指标筛选完成。")
    print(f"筛选后的数据包含 {selected_df.shape[0]} 行 和 {selected_df.shape[1]} 列。")

    # 4. 将筛选后的数据保存到新的Excel文件
    print(f"\n正在将筛选后的数据保存到: {full_output_path}")
    selected_df.to_excel(full_output_path, index=False) # index=False 避免将pandas的索引写入Excel
    print(f"数据已成功保存到 {full_output_path}")

except FileNotFoundError:
    print(f"错误: 输入文件未找到 '{full_input_path}'。请检查路径和文件名。")
except KeyError as e:
    # This specific KeyError for column not found during selection should be less likely
    # now due to the pre-check, but kept as a safeguard.
    print(f"错误: 在筛选过程中，原始文件中找不到指定的列: {e}。")
    print("这通常发生在 'columns_to_keep_actual' 列表中的列名与Excel中的标题在筛选前检查后仍不匹配（罕见）。")
except Exception as e:
    print(f"处理过程中发生了一个意料之外的错误: {e}")